## This script are an example in how to use the AppEARS API. Login, create token, submit a task, download the files from the task submited after its completed and save it. 

### First we make explicit the installation of libraries we will use and the importing.

In [1]:
!pip3 install -q --upgrade pip
!pip3 install -q pandas numpy matplotlib seaborn openpyxl climateserv requests netCDF4 xarray pyproj python-dotenv

In [2]:
import sys
from pathlib import Path

# Calculate the path to the scripts folder relative to the current notebook.
scripts_dir = Path("../../").resolve()

# Add the scripts directory to the sys.path if it's not already there.
if str(scripts_dir) not in sys.path:
    sys.path.append(str(scripts_dir))

In [3]:
import os
import json
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from typing import Dict, List, Tuple, Optional, Any


# Load environment variables from .env file
load_dotenv()

from scripts.preprocessing.api_appears import login, get_products, create_dataframe, submit_task, check_all_tasks_status
from scripts.preprocessing.api_appears import check_specific_task_status, create_and_submit_task, download_and_save_results
from scripts.preprocessing.api_appears import list_files_in_bundle, download_file_from_bundle, save_task_ids_to_csv, process_csv_and_submit_tasks

### Here below we make the login using our username and passwords, values that need to be confidencial. So we save this values into a .env file

In [4]:
# Example Usage
username = os.getenv('EARTHDATA_USERNAME')
password = os.getenv('EARTHDATA_PASSWORD')

# Login and get token
token = login(username=username, password=password)

token

'Z_4Y-vX8iITm4IJbGE9lCivZMq4nYrWz2k3L5qQhsLnNTvOWYlptLNAf0Q2R5DcpW23fQRQ4pEvGGfOIidpQOg'

### Here below we use a dataframe to extract all the points (latitude and longitude) we want to make a task (extract information from AppEARS products).

In [5]:
# Define the task parameters
task_type = 'point'  # Changed from 'area' to 'point'
task_name = 'NDVI-AquaViva-Point-250mts-16days-MOD13Q1'
start_date = '01-01-2015'  # Adjust as needed
end_date = '12-31-2022'    # Adjust as needed
product = 'MOD13Q1.061'
file_type = 'netcdf4'
# Since it's a point task, we won't use a GeoJSON object
geojson = None

# Create a list of layers as per your area task
layers = [
    "_250m_16_days_EVI",
    "_250m_16_days_NDVI",
    "_250m_16_days_MIR_reflectance",
    "_250m_16_days_VI_Quality",
    "_250m_16_days_composite_day_of_the_year",
    "_250m_16_days_pixel_reliability"
]

# Process the CSV and submit tasks
csv_file_path = '../../data/processed_data/igrac/wells_gambia.csv'
tasks = process_csv_and_submit_tasks(csv_file=csv_file_path, 
                                     username=username, 
                                     password=password, 
                                     task_type=task_type, 
                                     base_task_name=task_name, 
                                     start_date=start_date, 
                                     end_date=end_date, 
                                     product=product, 
                                     file_type=file_type, 
                                     layers=layers,
                                     geojson=geojson)

# Save the task IDs to a new CSV file
output_csv_path = '../../data/original_data/appears/ndvi/tasks_ndvi_wells_gambia.csv'
save_task_ids_to_csv(tasks, output_csv_path)

Submitted task for ID 0000058001, coordinates (13.4984817969726, -14.7573025092349). Task ID: 233a5ccc-7531-4885-b158-062b9b185481
Submitted task for ID 0000035001, coordinates (13.5172450917462, -15.242125038732). Task ID: bc3d7956-0bf3-415b-9201-b258585ea970
Submitted task for ID 0000024001, coordinates (13.2089634495222, -16.2989315774497). Task ID: dd52548a-8010-4405-a233-d8412eb51317
Submitted task for ID 0000023001, coordinates (13.2330569152907, -16.5047276118478). Task ID: d3582106-0e27-4c81-a710-4a1b9ff58387
Submitted task for ID 0000020007, coordinates (13.362761, -16.637219). Task ID: 1d92191d-5ffe-491c-beca-a6dc1d338e94
Submitted task for ID 0000020006, coordinates (13.3608101076666, -16.625421). Task ID: c7f74381-731a-4507-b236-fe377124444c
Submitted task for ID 0000022001, coordinates (13.3299965083864, -16.6587771128152). Task ID: 6d52a1e8-d83c-4619-a253-619a803de618
Submitted task for ID 0000020005, coordinates (13.3847523742244, -16.6797212636949). Task ID: 446723f0-a6

### Here below we can see all the files we can download from a particular task.

In [6]:
list_files_in_bundle(token=token, task_id='ef2d3522-219b-4b48-a21a-f9bfcfb3d0b0')

{'files': [{'sha256': 'a64d9a634559b107f798d463f5bd0ec57e455b1c2d4645793df17d4a9ffa7598',
   'file_id': '0dff60f9-8d60-4749-9095-b63f67319781',
   'file_name': 'ALL-API-NDVI-AquaViva-Point-250mts-16days-MOD13Q1-MOD13Q1-061-results.csv',
   'file_size': 71248,
   'file_type': 'csv',
   's3_url': 's3://appeears-output/ef2d3522-219b-4b48-a21a-f9bfcfb3d0b0/ALL-API-NDVI-AquaViva-Point-250mts-16days-MOD13Q1-MOD13Q1-061-results.csv'},
  {'sha256': '25044b2da1e7ad7a0806e206df36f9c9c5f652d143daddcd530455032ca78eb7',
   'file_id': 'e3023ab1-7f82-4325-bcb4-1b03ef8fa308',
   'file_name': 'ALL-API-NDVI-AquaViva-Point-250mts-16days-MOD13Q1-granule-list.txt',
   'file_size': 29970,
   'file_type': 'txt',
   's3_url': 's3://appeears-output/ef2d3522-219b-4b48-a21a-f9bfcfb3d0b0/ALL-API-NDVI-AquaViva-Point-250mts-16days-MOD13Q1-granule-list.txt'},
  {'sha256': '3f75ccf5a33bb54e3452e35194f845c398ba5a5dbe46379b34ee15944d68f30b',
   'file_id': '40be6438-8138-4e34-8fc6-51c05dc1c4e6',
   'file_name': 'ALL-API

### After all the tasks are ready to download we use the download_and_save_results to save it all the valuable files into a folder we specify

In [5]:
# Example usage
tasks_csv_path = '../../data/original_data/appears/ndvi/tasks_ndvi_wells_gambia.csv'  # Replace with your tasks CSV file path
destination_directory = '../../data/original_data/appears/ndvi'  # Replace with your desired directory path

download_and_save_results(tasks_csv=tasks_csv_path, destination_dir=destination_directory, token=token)


Processing Task ID: 233a5ccc-7531-4885-b158-062b9b185481 for Location ID: 0000058001
File downloaded successfully: ../../data/original_data/appears/ndvi/0000058001.csv
Downloaded file for Location ID: 0000058001 to ../../data/original_data/appears/ndvi/0000058001.csv
Processing Task ID: bc3d7956-0bf3-415b-9201-b258585ea970 for Location ID: 0000035001
File downloaded successfully: ../../data/original_data/appears/ndvi/0000035001.csv
Downloaded file for Location ID: 0000035001 to ../../data/original_data/appears/ndvi/0000035001.csv
Processing Task ID: dd52548a-8010-4405-a233-d8412eb51317 for Location ID: 0000024001
File downloaded successfully: ../../data/original_data/appears/ndvi/0000024001.csv
Downloaded file for Location ID: 0000024001 to ../../data/original_data/appears/ndvi/0000024001.csv
Processing Task ID: d3582106-0e27-4c81-a710-4a1b9ff58387 for Location ID: 0000023001
File downloaded successfully: ../../data/original_data/appears/ndvi/0000023001.csv
Downloaded file for Location

## Finally we will have all our data ready to pre-process and use for our satelital project.